In [1]:
import os
import numpy as np
import tensorflow as tf
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
#from PIL import Image
import cv2 as cv
import imutils
model_file1 = "./trans_model/tf_model.pb"
model_file2 = "./trans_model/frozen_onet_model.pb"
#model_path = "./frozentensorflowModel.pb"
#pbtxt_path = "/workspace/testbuildmodel/glassdemo/dataset/glassdemo.pbtxt"



In [2]:
#testimg = "/workspace/testbuildmodel/glassdemo/dataset/Folder/lighter_video/1/out155.jpeg"
testvideo= "/workspace/testbuildmodel/glassdemo/dataset/Folder/lighter_video/1/lighter_1.mp4"

In [3]:
"""
cap = cv.VideoCapture(0)
while(True):
    print("fefe0")
  # 從攝影機擷取一張影像
    ret, frame = cap.read()
    print("fefe1")
  # 顯示圖片
    cv.imshow('frame', frame)
    print("fefe2")
  # 若按下 q 鍵則離開迴圈
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv.destroyAllWindows()
"""

'\ncap = cv.VideoCapture(0)\nwhile(True):\n    print("fefe0")\n  # \xe5\xbe\x9e\xe6\x94\x9d\xe5\xbd\xb1\xe6\xa9\x9f\xe6\x93\xb7\xe5\x8f\x96\xe4\xb8\x80\xe5\xbc\xb5\xe5\xbd\xb1\xe5\x83\x8f\n    ret, frame = cap.read()\n    print("fefe1")\n  # \xe9\xa1\xaf\xe7\xa4\xba\xe5\x9c\x96\xe7\x89\x87\n    cv.imshow(\'frame\', frame)\n    print("fefe2")\n  # \xe8\x8b\xa5\xe6\x8c\x89\xe4\xb8\x8b q \xe9\x8d\xb5\xe5\x89\x87\xe9\x9b\xa2\xe9\x96\x8b\xe8\xbf\xb4\xe5\x9c\x88\n    if cv.waitKey(1) & 0xFF == ord(\'q\'):\n        break\n\n# \xe9\x87\x8b\xe6\x94\xbe\xe6\x94\x9d\xe5\xbd\xb1\xe6\xa9\x9f\ncap.release()\n\n# \xe9\x97\x9c\xe9\x96\x89\xe6\x89\x80\xe6\x9c\x89 OpenCV \xe8\xa6\x96\xe7\xaa\x97\ncv.destroyAllWindows()\n'

In [4]:
"""
with tf.gfile.FastGFile(model_path, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Session() as sess:
    # Restore session
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
"""

"\nwith tf.gfile.FastGFile(model_path, 'rb') as f:\n    graph_def = tf.GraphDef()\n    graph_def.ParseFromString(f.read())\n\nwith tf.Session() as sess:\n    # Restore session\n    sess.graph.as_default()\n    tf.import_graph_def(graph_def, name='')\n"

In [5]:
def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

In [6]:
classify_graph1 = load_graph(model_file1)
sess1=tf.Session(graph=classify_graph1)

classify_graph2 = load_graph(model_file2)
sess2=tf.Session(graph=classify_graph2)

In [7]:
#for op in tf.get_default_graph().get_operations():
    #print(str(op.name))

In [8]:
#sess2.graph.get_operations()

In [9]:
"""
import time

with tf.Session() as sess:
    # Restore session

    #alltensor=sess.graph.get_operations()
    #print(alltensor)
    cap = cv.VideoCapture(0)
    cap.set(cv.CAP_PROP_FRAME_WIDTH, 1280);
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720);
    while(True):
        
       
        ret, img = cap.read()
        # Read and preprocess an image.
        #img = cv.imread(testimg)
        rows = img.shape[0]
        cols = img.shape[1]
        img2=img.copy()
        inp = cv.resize(img2, (300, 300))
        #inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

        # Run the model
        start_time = time.time()
        print("FFFFF")
        out = sess.run([sess.graph.get_tensor_by_name('claasf1_1/dense_1/BiasAdd:0'),],
                       feed_dict={'Reshape:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})
        print("FFFFF")
        elapsed_time = time.time() - start_time
        print(elapsed_time)
        predctedArray=out[0][0].astype(int)
        
        # Visualize detected bounding boxes.

        x1=max(0, min(predctedArray[2]-179, 319))
        if(x1!=319):
            y1=max(0, min(predctedArray[2]-x1, 179))
        else:
            y1=max(0, min(179-(predctedArray[2]-319-179), 179))

        x2=max(0, min(predctedArray[3]-179, 319))
        if(x2!=319):
            y2=max(0, min(predctedArray[3]-x1, 179))
        else:
            y2=max(0, min(179-(predctedArray[2]-319-179), 179))  
        #print(rows)
        #print(cols)
        img=cv.resize(img, (320, 180))
        cv.line(img, (predctedArray[0],predctedArray[1]), ( x1,y1), (0,0,255), thickness=1, lineType=8,  shift=0)
        cv.line(img, (predctedArray[0],predctedArray[1]), ( x2,y2),(0,0,255), thickness=1, lineType=8,  shift=0)

        #cv.putText(img, text, (10, 80), cv.FONT_HERSHEY_PLAIN,1, (0, 255, 255), 1, cv.LINE_AA)

                #print(classId, "-->", score, x, y)
        cv.imshow('frame', img)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()


cv.destroyAllWindows()
#cv.imshow('SHOWfg', imutils.resize(img, width=800))
#cv.waitKey()
"""

'\nimport time\n\nwith tf.Session() as sess:\n    # Restore session\n\n    #alltensor=sess.graph.get_operations()\n    #print(alltensor)\n    cap = cv.VideoCapture(0)\n    cap.set(cv.CAP_PROP_FRAME_WIDTH, 1280);\n    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720);\n    while(True):\n        \n       \n        ret, img = cap.read()\n        # Read and preprocess an image.\n        #img = cv.imread(testimg)\n        rows = img.shape[0]\n        cols = img.shape[1]\n        img2=img.copy()\n        inp = cv.resize(img2, (300, 300))\n        #inp = inp[:, :, [2, 1, 0]]  # BGR2RGB\n\n        # Run the model\n        start_time = time.time()\n        print("FFFFF")\n        out = sess.run([sess.graph.get_tensor_by_name(\'claasf1_1/dense_1/BiasAdd:0\'),],\n                       feed_dict={\'Reshape:0\': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})\n        print("FFFFF")\n        elapsed_time = time.time() - start_time\n        print(elapsed_time)\n        predctedArray=out[0][0].astype(int)\n   

In [10]:
#input_1_2:0
#dense_1_2/BiasAdd:0


In [ ]:
import time


    # Restore session

    #alltensor=sess.graph.get_operations()
    #print(alltensor)
cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 1280);
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720);
while(True):

    print(cap)
    ret, img = cap.read()
    # Read and preprocess an image.
    #img = cv.imread(testimg)
    rows = img.shape[0]
    cols = img.shape[1]
    img2=img.copy()
    midpint=cols/2
    print (img2.shape)
    img2 = img2[:, midpint-rows/2:midpint+rows/2,:]
    print(img2.shape)
    inp = cv.resize(img2, (100, 100))
    inp2 = cv.resize(img2, (48, 48))
    #inp2 = inp2[:, :, [2, 1, 0]]
    inp2=(inp2 - 127.5) * 0.0078125
    #print(in2p)
    #inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

    # Run the model
    start_time = time.time()

    out2 = sess2.run([sess2.graph.get_tensor_by_name('import/onet/conv6-2/onet/conv6-2:0'),sess2.graph.get_tensor_by_name('import/onet/conv6-3/onet/conv6-3:0')],
       feed_dict={'import/Placeholder:0': inp2.reshape(1, inp2.shape[0], inp2.shape[1], 3)})
    elapsed_time1 = time.time() - start_time
    
    start_time = time.time()
    out1 = sess1.run([sess1.graph.get_tensor_by_name('import/dense_1/BiasAdd:0'),],
                   feed_dict={'import/input_1:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})
    



    elapsed_time2 = time.time() - start_time
    print(elapsed_time1)
    print(elapsed_time2)
    predctedArray=out1[0][0]
    print('############')
    predctedArray2=out2[1]
    predctedArray2=(predctedArray2+1)/2
    predctedArray2=predctedArray2[0]
    print(predctedArray2.shape)
    print(predctedArray2)
    print('############')
    # Visualize detected bounding boxes.


    #print(predctedArray)
    #print(rows)
    #print(cols)
    img=cv.resize(img2, (300, 300))
    cv.ellipse(img, (int(predctedArray[0]*300),int(predctedArray[1]*300)), (20, 20), 0, 0, 0, (205, 0, 255), 10)
    cv.ellipse(img, (int(predctedArray[2]*300),int(predctedArray[3]*300)), (20, 20), 0, 0, 0, (205, 0, 255), 10)
    cv.ellipse(img, (int(predctedArray[4]*300),int(predctedArray[5]*300)), (20, 20), 0, 0, 0, (205, 0, 255), 10)
    cv.ellipse(img, (int(predctedArray[6]*300),int(predctedArray[7]*300)), (20, 20), 0, 0, 0, (205, 0, 255), 10)
    cv.ellipse(img, (int(predctedArray[8]*300),int(predctedArray[9]*300)), (20, 20), 0, 0, 0, (205, 0, 255), 10)
   
    cv.ellipse(img, (int(predctedArray2[0]*300),int(predctedArray2[1]*300)), (20, 20), 0, 0, 0, (205, 0, 0), 10)
    cv.ellipse(img, (int(predctedArray2[2]*300),int(predctedArray2[3]*300)), (20, 20), 0, 0, 0, (205, 0, 0), 10)
    cv.ellipse(img, (int(predctedArray2[4]*300),int(predctedArray2[5]*300)), (20, 20), 0, 0, 0, (205, 0, 0), 10)
    cv.ellipse(img, (int(predctedArray2[6]*300),int(predctedArray2[7]*300)), (20, 20), 0, 0, 0, (205, 0, 0), 10)
    cv.ellipse(img, (int(predctedArray2[8]*300),int(predctedArray2[9]*300)), (20, 20), 0, 0, 0, (205, 0, 0), 10)
    #cv.putText(img, text, (10, 80), cv.FONT_HERSHEY_PLAIN,1, (0, 255, 255), 1, cv.LINE_AA)

            #print(classId, "-->", score, x, y)
    cv.imshow('frame', img)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()


cv.destroyAllWindows()
#cv.imshow('SHOWfg', imutils.resize(img, width=800))
#cv.waitKey()